In [59]:
from dataclasses import dataclass
from typing import List, Protocol, Optional, Self
from abc import ABC, abstractmethod

@dataclass
class SerialSettings:
    port: str
    baudrate: 9600 | 115200
    timeout: int


class SerialLifecycle(Protocol):
    def onConnect(self):
        pass
    
    def onDisconnect(self):
        pass

    def onTimeout(self):
        pass

    def onAck(self):
        pass


class SerialConnection(ABC):
    @abstractmethod
    def __init__(self, settings: SerialSettings, lifecycle: Optional[SerialLifecycle]) -> Self:
        raise NotImplementedError()

    @abstractmethod
    def write(self, data: bytes) -> None:
        raise NotImplementedError()

    @abstractmethod
    def write_ack(self) -> None:
        raise NotImplementedError()

    @abstractmethod
    def read(self, size: int) -> bytes:
        raise NotImplementedError()

    @abstractmethod
    def readline(self) -> bytes:
        raise NotImplementedError()

    def onConnect(self) -> None:
        serialLifecycle.onConnect() if serialLifecycle else None
    
    def onDisconnect(self) -> None:
        serialLifecycle.onDisconnect() if serialLifecycle else None

    def onTimeout(self) -> None:
        serialLifecycle.onTimeout() if serialLifecycle else None

    def onAck(self) -> None:
        serialLifecycle.onAck() if serialLifecycle else None


class SerialConnectionMock(SerialConnection):
    def __init__(self, settings: SerialSettings, lifecycle: Optional[SerialLifecycle]):
        self._buffer: bytes = b''

    def write(self, data: bytes):
        self._buffer += data

    def write_ack(self):
        self.write(b'ACK')
        self.flush()
        if self.read(3) != b'ACK':
          raise Exception('ACK not received')
        self.onAck()

    def read(self, size: int):
        data = self._buffer[:size]
        self._buffer = self._buffer[size:]
        return data


serial = SerialMock(settings=SerialSettingsMock(port="COM1", baudrate=9600, timeout=1))
serial.write(b'Hello\nWorld\n')
serial.read(4)
# word1 = serial.readline()
# word2 = serial.readline()
# print(word1, word2)

b'Hell'